In [1]:
import ee

In [2]:
ee.Initialize()

### Define premise parameter

In [3]:
# define the year range
year_start = [f'{i}-01-01' for i in range(1987,2020,3)]
year_end   = [f'{i}-12-31' for i in range(1989,2020,3)]

year_range = list(zip(year_start,year_end))

# import the research area boundary
North_China_Plain = ee.FeatureCollection("users/wangjinzhulala/North_China_Plain/0_1_North_China_Plain_Full")

In [4]:
year_range

[('1987-01-01', '1989-12-31'),
 ('1990-01-01', '1992-12-31'),
 ('1993-01-01', '1995-12-31'),
 ('1996-01-01', '1998-12-31'),
 ('1999-01-01', '2001-12-31'),
 ('2002-01-01', '2004-12-31'),
 ('2005-01-01', '2007-12-31'),
 ('2008-01-01', '2010-12-31'),
 ('2011-01-01', '2013-12-31'),
 ('2014-01-01', '2016-12-31'),
 ('2017-01-01', '2019-12-31')]

### Select Correct Landsat img and Compute the Mean-NDVI according to year_range

In [5]:
# use if condition to find out the right Landsat img formulation
# here use Landsat Raw data

Mean_img = []

for span in year_range:
    
    # find the end year to determine the 
    start_year = int(span[0].split('-')[0])
    end_year   = int(span[1].split('-')[0])
    range_name = f'{start_year}_{end_year}'

    if  end_year  <= 2010:
        NDVI_img = ee.ImageCollection("LANDSAT/LT05/C01/T1").select(['B4','B3'])\
                        .filterBounds(North_China_Plain).filterDate(span[0],span[1])\
                        .mean()\
                        .normalizedDifference()\
                        .clip(North_China_Plain)

    elif end_year <= 2013:
        NDVI_img = ee.ImageCollection("LANDSAT/LE07/C01/T1").select(['B4','B3'])\
                        .filterBounds(North_China_Plain).filterDate(span[0],span[1])\
                        .mean()\
                        .normalizedDifference()\
                        .clip(North_China_Plain)

    else:
        NDVI_img = ee.ImageCollection("LANDSAT/LC08/C01/T1").select(['B5','B4'])\
                        .filterBounds(North_China_Plain).filterDate(span[0],span[1])\
                        .mean()\
                        .normalizedDifference()\
                        .clip(North_China_Plain)
    
    # add the range_name and landsat img to the list
    Mean_img.append((range_name,NDVI_img))

### Export the mean-NDVI img

In [7]:
for span_range,ndvi in Mean_img:
    
    # define the GEE_asset path
    path = 'users/wensomone666/Jinzhu/Mean_NDVI'
    # define the name
    name = f'Year_{span_range}_Mean_NDVI'

    # multiply the NDVI with 100 and change it 
    # into a int-img to reduce space
    img = ndvi.multiply(100).toInt8()

    # export
#     task = ee.batch.Export.image.toAsset(   image           = img,
#                                             description     = name,
#                                             assetId         = f'{path}/{name}',
#                                             region          = North_China_Plain.geometry().bounds(),
#                                             scale           = 30,
#                                             maxPixels       = int(1e13)
#                                         )

#     task.start()

    # print out the process
    print(f'Exporting mean-NDVI of {name}')

Exporting mean-NDVI of Year_1987_1989_Mean_NDVI
Exporting mean-NDVI of Year_1990_1992_Mean_NDVI
Exporting mean-NDVI of Year_1993_1995_Mean_NDVI
Exporting mean-NDVI of Year_1996_1998_Mean_NDVI
Exporting mean-NDVI of Year_1999_2001_Mean_NDVI
Exporting mean-NDVI of Year_2002_2004_Mean_NDVI
Exporting mean-NDVI of Year_2005_2007_Mean_NDVI
Exporting mean-NDVI of Year_2008_2010_Mean_NDVI
Exporting mean-NDVI of Year_2011_2013_Mean_NDVI
Exporting mean-NDVI of Year_2014_2016_Mean_NDVI
Exporting mean-NDVI of Year_2017_2019_Mean_NDVI
